<a href="https://colab.research.google.com/github/nkcong206/Travel-Recommendation-System/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install -q langchain-ollama
!pip install -q langchain
!pip install -q langchain_community
!pip install -q sentence-transformers
!pip install -qU langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 8.1 MB/s eta 0:00:00


In [135]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from sentence_transformers import SentenceTransformer
from langchain_google_genai import ChatGoogleGenerativeAI
import re
import json
import subprocess
import time
import threading
import psycopg2
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [126]:
import getpass
import os

postgres_url = getpass.getpass("Enter your postgresql url: ")

Enter your postgresql url: ··········


In [31]:
model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

### LLama 3

In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Installing ollama to /usr/local
100 13344    0 13344    0     0  62077      0 --:--:-- --:--:-- --:--:-- 62355
>>> Downloading Linux amd64 CLI
############################################################################################# 100.0%
>>> Making ollama accessible in the PATH in /usr/local/bin
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# Start the ollama server in a new process
process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Function to print server output
def print_output(process):
    while True:
        output = process.stdout.readline()
        if output == b'' and process.poll() is not None:
            break
        if output:
            print(output.strip().decode('utf-8'))
        time.sleep(1)

# Start a thread to print server output
thread = threading.Thread(target=print_output, args=(process,))
thread.start()

print("Ollama server is running in the background")

Ollama server is running in the background


In [ ]:
!ollama pull llama3:latest

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is:

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIJFZhADPB4fUDfhLGKqOCrM2vPlYMgu8lGG3uJho+tHE

[GIN] 2024/08/20 - 14:50:25 | 200 |     123.403µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏ 6.7 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  30 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  50 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  58 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   2% ▕▏  72 MB/4.7 GB                  pulling manifest 
pulling 

In [ ]:
llm = OllamaLLM(model="llama3:latest")

### Gemini

In [136]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [137]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

### Prompt để lấy thông tin

In [153]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("SET search_path TO travel_database, public;")

cur.execute("""
    SELECT DISTINCT unnest(amenities) AS unique_amenity
    FROM hotel;
""")

rows = cur.fetchall()

cur.close()
conn.close()

amenities_list = [row[0] for row in rows]

amenities_list_str = "\n    ".join(f'{amenity},' for amenity in amenities_list)
print(amenities_list_str)

Truyền hình cáp,
    Minibar,
    Nhân viên xách hành lý,
    Bồn tắm,
    Wifi,
    Dịch vụ giặt ủi,
    Thang máy,
    Đưa đón sân bay,
    Bữa sáng,
    Thức uống chào mừng miễn phí,
    Mát-xa,
    Bữa sáng món tự chọn,
    Khu vực hút thuốc,
    Dịch vụ lưu trữ hành lý,


In [154]:
style_list = ["Luxury", "Eco-friendly", "Boutique", "Beachfront", "Family-friendly", "Business", "Romantic", "Cultural", "Wellness", "Adventure"]
style_list_str = "\n    ".join(f'{style},' for style in style_list)
print(style_list_str)

Luxury,
    Eco-friendly,
    Boutique,
    Beachfront,
    Family-friendly,
    Business,
    Romantic,
    Cultural,
    Wellness,
    Adventure,


In [171]:
template = """
    You are an AI travel suggestion chatbot. Your task is to analyze the Request:

    Request: "{travel_request}"

    Extract the following 7 pieces of information not in any specific order: Type, District, City, Number_of_people, Price, Rating, and Description.
    Also, identify any amenities and a single style mentioned in the request from the following lists:

    Amenities:
    "{amenities_list_str}"

    Styles:
    "{style_list_str}"

    Return the result in Vietnamese in the following format:
    {{
        "Type": "...",
        "District": "...",
        "City": "...",
        "Number_of_people": "...",
        "Price": "...",
        "Rating": "...",
        "Description": "...",
        "Amenities": [...],
        "Style": "..."
    }}

    Ensure that the "Type" is one of the following: "Hotel," "Restaurant," or "TouristAttraction." If one or two types are mentioned, return only those. If none are mentioned, include all three types.

    For any information not specified in the travel request, return `null`. If no style is mentioned, return `"none"`. Ensure that the JSON result is strictly valid JSON, with no extra text, comments, or parentheses.

    The "District" should not include "quận" or "huyện". The "City" should not include "thành phố" or "tỉnh".

    The "Price" should be an integer. The "Rating" should be a float.
"""
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm

In [172]:
query = """
Gợi ý cho tôi một lộ trình du lịch tại Hà Nội với khách sạn sang trọng, có bể bơi và spa, nhà hàng phục vụ món ăn truyền thống và một điểm tham quan nổi tiếng.
Tôi muốn biết thêm về các tiện nghi của khách sạn và phong cách của nhà hàng."""
response = chain.invoke(
{
    "travel_request":query,
    "amenities_list_str": amenities_list_str,
    "style_list_str": style_list_str
}
)
response_gemini = str(response.content)
cleaned_json_str = re.search(r'\{.*?\}', response_gemini, re.DOTALL).group(0)
result_dict = json.loads(cleaned_json_str)
result_dict

{'Type': ['Hotel', 'Restaurant', 'TouristAttraction'],
 'District': 'null',
 'City': 'Hà Nội',
 'Number_of_people': 'null',
 'Price': 'null',
 'Rating': 'null',
 'Description': 'Gợi ý cho tôi một lộ trình du lịch tại Hà Nội với khách sạn sang trọng, có bể bơi và spa, nhà hàng phục vụ món ăn truyền thống và một điểm tham quan nổi tiếng.',
 'Amenities': ['Bể bơi', 'Spa'],
 'Style': 'Luxury'}

In [27]:
def get_embedding(text):
    """Trả về embedding của văn bản sử dụng mô hình SentenceTransformer."""
    embedding = model.encode(text)
    return embedding.tolist()

In [ ]:
# Kết nối với cơ sở dữ liệu PostgreSQL
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Lấy dữ liệu tiện nghi của các khách sạn có tên là "A"
cur.execute("""
    SELECT hotel_id, rating, amenities, style, description
    FROM hotel
    WHERE 'Wifi' = ANY(amenities);
""")

# Chuyển đổi dữ liệu thành DataFrame
data = cur.fetchall()
df = pd.DataFrame(data, columns=['hotel_id', 'rating','amenities', 'style', 'description'])

cur.close()

In [114]:
data = [
    [1,1, 4.5, ['WiFi', 'Pool', 'Spa'], 'Luxury', 'A beautiful luxury hotel with a spa and pool.', 90],
    [2,1, 4.5, ['WiFi', 'Pool', 'Spa'], 'Luxury', 'A beautiful luxury hotel with a spa and pool.', 100],
    [3,2, 2.3, ['WiFi', 'Gym', 'Spa'], 'Eco-friendly', 'An eco-friendly hotel with gym facilities.', 80],
    [4,3, 1.2, ['WiFi', 'Beach'], 'Beachfront', 'A beachfront hotel with stunning ocean views.', 70]
]

df_data = pd.DataFrame(data, columns=['id', 'hotel_id', 'rating', 'amenities', 'style', 'description', 'price'])

query = [[  5, ['WiFi', 'Pool', 'Spa', 'Beach'], 'Luxury', 'A beautiful luxury hotel.', 110]]

df_query = pd.DataFrame(query, columns=[ 'rating', 'amenities', 'style', 'description', 'price'])

df = pd.concat([df_query, df_data], ignore_index=True)

df_amenities = df[['id', 'amenities']].copy()
df_style = df[['id', 'style']].copy()
df_description = df[['id', 'description']].copy()
df_rating = df[['id', 'rating']].copy()
df_price = df[['id', 'price']].copy()
# Sử dụng MultiLabelBinarizer để one-hot encode cột amenities
mlb = MultiLabelBinarizer()
amenities_encoded = mlb.fit_transform(df_amenities['amenities'])

# Thay thế cột amenities bằng các vector one-hot encoded
df_amenities['amenities_nor'] = amenities_encoded.tolist()

# Hiển thị kết quả
print(df_amenities)

    id                 amenities    amenities_nor
0  NaN  [WiFi, Pool, Spa, Beach]  [1, 0, 1, 1, 1]
1  1.0         [WiFi, Pool, Spa]  [0, 0, 1, 1, 1]
2  2.0         [WiFi, Pool, Spa]  [0, 0, 1, 1, 1]
3  3.0          [WiFi, Gym, Spa]  [0, 1, 0, 1, 1]
4  4.0             [WiFi, Beach]  [1, 0, 0, 0, 1]


In [115]:
similarities = [cosine_similarity([df_amenities['amenities_nor'][0]], [amenity_vector])[0][0]
                for amenity_vector in df_amenities['amenities_nor']]

# Thêm cột cosine similarity vào df_amenities
df_amenities['amenities_similarity'] = similarities

# Hiển thị kết quả
print(df_amenities)

    id                 amenities    amenities_nor  amenities_similarity
0  NaN  [WiFi, Pool, Spa, Beach]  [1, 0, 1, 1, 1]              1.000000
1  1.0         [WiFi, Pool, Spa]  [0, 0, 1, 1, 1]              0.866025
2  2.0         [WiFi, Pool, Spa]  [0, 0, 1, 1, 1]              0.866025
3  3.0          [WiFi, Gym, Spa]  [0, 1, 0, 1, 1]              0.577350
4  4.0             [WiFi, Beach]  [1, 0, 0, 0, 1]              0.707107


In [116]:
df_description['description_embedding'] = df_description['description'].apply(get_embedding)
# Tính cosine similarity giữa mỗi embedding và embedding của hàng 0
similarities = [cosine_similarity([df_description['description_embedding'][0]], [embedding])[0][0]
                for embedding in df_description['description_embedding']]

# Thêm cột cosine similarity vào df_description
df_description['description_similarity'] = similarities

# Hiển thị kết quả
print(df_description)

    id                                    description  \
0  NaN                      A beautiful luxury hotel.   
1  1.0  A beautiful luxury hotel with a spa and pool.   
2  2.0  A beautiful luxury hotel with a spa and pool.   
3  3.0     An eco-friendly hotel with gym facilities.   
4  4.0  A beachfront hotel with stunning ocean views.   

                               description_embedding  description_similarity  
0  [0.09188824146986008, -0.21028681099414825, -0...                1.000000  
1  [0.09868957102298737, -0.14982357621192932, -0...                0.817774  
2  [0.09868957102298737, -0.14982357621192932, -0...                0.817774  
3  [0.29612961411476135, -0.10158709436655045, -0...                0.323857  
4  [-0.05280661582946777, -0.21619683504104614, 0...                0.215760  


In [122]:
# scaler = MinMaxScaler()
# df_rating['rating_similarity'] = scaler.fit_transform(df_rating[['rating']])

max_value = df_rating['rating'].max()

df_rating['rating_similarity'] =  df_rating['rating']  / max_value

print(df_rating)

    id  rating  rating_similarity
0  NaN     5.0               1.00
1  1.0     4.5               0.90
2  2.0     4.5               0.90
3  3.0     2.3               0.46
4  4.0     1.2               0.24


In [123]:
# scaler = MinMaxScaler()

# df_price['price_similarity'] = scaler.fit_transform(df_price[['price']])

max_value = df_price['price'].max()

df_price['price_similarity'] =  df_price['price']  / max_value

print(df_price)

    id  price  price_similarity
0  NaN    110          1.000000
1  1.0     90          0.818182
2  2.0    100          0.909091
3  3.0     80          0.727273
4  4.0     70          0.636364


In [125]:
df_ranking = df[['id']].copy()
df_ranking = df_ranking.merge(df_amenities[['id', 'amenities_similarity']], on='id', how='left')
df_ranking = df_ranking.merge(df_description[['id', 'description_similarity']], on='id', how='left')
df_ranking = df_ranking.merge(df_rating[['id', 'rating_similarity']], on='id', how='left')
df_ranking = df_ranking.merge(df_price[['id', 'price_similarity']], on='id', how='left')
df_ranking['total_similarity'] = (df_ranking['amenities_similarity'] +
                                  df_ranking['description_similarity'] +
                                  df_ranking['rating_similarity'] -
                                  2*df_ranking['price_similarity'])
# Hiển thị kết quả
df_ranking = df_ranking.iloc[1:].reset_index(drop=True)
print(df_ranking)
# Tìm giá trị lớn nhất trong cột 'total_similarity'
max_total_similarity_id = df_ranking.loc[df_ranking['total_similarity'].idxmax(), 'id']

print("1st: ",max_total_similarity_id)

    id  amenities_similarity  description_similarity  rating_similarity  \
0  1.0              0.866025                0.817774               0.90   
1  2.0              0.866025                0.817774               0.90   
2  3.0              0.577350                0.323857               0.46   
3  4.0              0.707107                0.215760               0.24   

   price_similarity  total_similarity  
0          0.818182          0.947436  
1          0.909091          0.765617  
2          0.727273         -0.093338  
3          0.636364         -0.109860  
1st:  1.0


### Run nếu dùng LLama 3

In [ ]:
query = """
Gợi ý cho tôi 1 khách sạn ở Cầu Giấy, Hà Nội với chất lượng dịch vụ cao cấp!
"""
response = chain.invoke(query)
cleaned_json_str = re.search(r'\{.*?\}', response, re.DOTALL).group(0)
result_dict = json.loads(cleaned_json_str)
result_dict

### Run nếu dùng Gemini

In [ ]:
query = """
Gợi ý cho tôi 1 khách sạn ở Cầu Giấy, Hà Nội với chất lượng dịch vụ cao cấp!
"""
response = chain.invoke(query)
response_gemini = str(response.content)
cleaned_json_str = re.search(r'\{.*?\}', response_gemini, re.DOTALL).group(0)
result_dict = json.loads(cleaned_json_str)
result_dict

{'Type': 'Hotel',
 'District': 'Cầu Giấy',
 'City': 'Hà Nội',
 'Number_of_people': None,
 'Price': None,
 'Rating': None,
 'Description': 'chất lượng dịch vụ cao cấp'}

In [ ]:
# Gợi ý cho tôi khách sạn ở Cầu Giấy, Hà Nội chất lượng 5 sao giành cho 2 người giá khoảng 10 triệu đồng với chất lượng dịch vụ cao cấp!
query = input("Nhập câu truy vấn của bạn: ")

Nhập câu truy vấn của bạn: Gợi ý cho tôi khách sạn ở Cầu Giấy, Hà Nội chất lượng 5 sao giành cho 2 người giá khoảng 10 triệu đồng với chất lượng dịch vụ cao cấp!


In [ ]:
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Đặt schema hiện tại
cur.execute("SET search_path TO travel_database, public;")

# Tạo vector cho mô tả truy vấn


# Thành phố cần tìm
response = chain.invoke(query)
cleaned_json_str = re.search(r'\{.*?\}', response, re.DOTALL).group(0)
result_dict = json.loads(cleaned_json_str)


embedding_query = model.encode(result_dict['Description'])
embedding_query_list = embedding_query.tolist()

# Kiểm tra xem bảng Hotel có tồn tại không
cur.execute("SELECT * FROM information_schema.tables WHERE table_name = 'hotel';")
if not cur.fetchone():
    print("Bảng 'Hotel' không tồn tại trong schema 'travel_database'.")
else:
    # Thực hiện truy vấn để tìm 2 khách sạn có độ tương đồng cao nhất
    cur.execute("""
        WITH query_embedding AS (
            SELECT %s::vector(768) AS query_vector
        ),
        similarity_scores AS (
            SELECT
                h.hotel_id,
                h.name,
                h.address,
                h.rating,
                h.description,
                (1 - (h.embedding_description <=> query_vector)) AS similarity
            FROM
                Hotel h
                CROSS JOIN query_embedding
            WHERE
                (h.address).city = %s
            ORDER BY
                similarity DESC
            LIMIT 2
        )
        -- Chọn 2 khách sạn có độ tương đồng cao nhất
        SELECT
            hotel_id,
            name,
            address,
            rating,
            description,
            similarity
        FROM
            similarity_scores;
    """, (embedding_query_list, result_dict['City']))

    # Lấy kết quả và in ra
    rows = cur.fetchall()

    print(f"Yêu cầu người dùng: {query}")
    for row in rows:
        print(f"HotelID: {row[0]}")
        print(f"Name: {row[1]}")
        print(f"Address: {row[2]}")
        print(f"Rating: {row[3]}")
        print(f"Description: {row[4]}")
        print(f"Similarity: {row[5]}")
        print("-" * 40)

    # print(result_dict)
# Đóng kết nối
cur.close()
conn.close()

Yêu cầu người dùng: Khách sạn giá rẻ nhưng chất lượng tốt, phù hợp cho các du khách với ngân sách hạn chế.
HotelID: 3
Name: Khách sạn C
Address: (0,"Cầu Giấy","Hà Nội")
Rating: 4.2
Similarity: 1.0
----------------------------------------
HotelID: 8
Name: Khách sạn C
Address: (0,"Cầu Giấy","Hà Nội")
Rating: 4.2
Similarity: 1.0
----------------------------------------


In [ ]:
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Đặt schema hiện tại
cur.execute("SET search_path TO travel_database, public;")

# Tạo vector cho mô tả truy vấn


# Thành phố cần tìm
response = chain.invoke(query)
response_gemini = str(response.content)
cleaned_json_str = re.search(r'\{.*?\}', response_gemini, re.DOTALL).group(0)
result_dict = json.loads(cleaned_json_str)


embedding_query = model.encode(result_dict['Description'])
embedding_query_list = embedding_query.tolist()

# Kiểm tra xem bảng Hotel có tồn tại không
cur.execute("SELECT * FROM information_schema.tables WHERE table_name = 'hotel';")
if not cur.fetchone():
    print("Bảng 'Hotel' không tồn tại trong schema 'travel_database'.")
else:
    # Thực hiện truy vấn để tìm 2 khách sạn có độ tương đồng cao nhất
    cur.execute("""
        WITH query_embedding AS (
            SELECT %s::vector(768) AS query_vector
        ),
        similarity_scores AS (
            SELECT
                h.hotel_id,
                h.name,
                h.address,
                h.rating,
                h.description,
                (1 - (h.embedding_description <=> query_vector)) AS similarity
            FROM
                Hotel h
                CROSS JOIN query_embedding
            WHERE
                (h.address).city = %s
            ORDER BY
                similarity DESC
            LIMIT 2
        )
        -- Chọn 2 khách sạn có độ tương đồng cao nhất
        SELECT
            hotel_id,
            name,
            address,
            rating,
            description,
            similarity
        FROM
            similarity_scores;
    """, (embedding_query_list, result_dict['City']))

    # Lấy kết quả và in ra
    rows = cur.fetchall()

    print(f"Yêu cầu người dùng: {query}")
    for row in rows:
        print(f"HotelID: {row[0]}")
        print(f"Name: {row[1]}")
        print(f"Address: {row[2]}")
        print(f"Rating: {row[3]}")
        print(f"Description: {row[4]}")
        print(f"Similarity: {row[5]}")
        print("-" * 40)

    print(result_dict)
# Đóng kết nối
cur.close()
conn.close()

Yêu cầu người dùng: 
Gợi ý cho tôi 1 khách sạn ở Cầu Giấy, Hà Nội với chất lượng dịch vụ cao cấp!

HotelID: 1
Name: Khách sạn A
Address: (0,"Cầu Giấy","Hà Nội")
Rating: 4.5
Description: Khách sạn cao cấp gần trung tâm Hà Nội, với các tiện nghi hiện đại và dịch vụ khách hàng xuất sắc.
Similarity: 0.23988989521233273
----------------------------------------
HotelID: 4
Name: Khách sạn D
Address: (0,"Cầu Giấy","Hà Nội")
Rating: 4.9
Description: Khách sạn sang trọng với dịch vụ 5 sao, bao gồm spa, bể bơi và nhà hàng cao cấp.
Similarity: 0.19820067049122325
----------------------------------------
{'Type': 'Hotel', 'District': 'Cầu Giấy', 'City': 'Hà Nội', 'Number_of_people': None, 'Price': None, 'Rating': None, 'Description': 'chất lượng dịch vụ cao cấp'}
